In [1]:
import numpy as np
import pandas as pd

In [2]:
X = np.load('/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/data/X.npy',allow_pickle= True)
y = np.load('/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/data/y.npy',allow_pickle= True)

In [3]:
X.shape

(41157,)

In [4]:
from Modules.gensim_vectorizers import FastText_vectorize
vec = FastText_vectorize("/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/fasttext/wiki-news-300d-1M.vec")

In [5]:
vec._build_vector()

999995it [01:43, 9621.00it/s] 


In [6]:
X, y = X[:3500] , y[:3500]

In [7]:
print(X.shape, y.shape)

(3500,) (3500,)


In [8]:
pd.DataFrame(y).value_counts()

2    1461
0    1434
1     605
dtype: int64

In [9]:
# vec_list = [vec._get_vector_list(x).tolist() for x in X]

In [10]:
# len(vec_list)

In [11]:
# vec_list = [vec.padding_truncate_lists(vec_list[x],max_length=20) for x in range(len(vec_list))]

In [12]:
vec_list = np.array([vec.padding_truncate(vec._get_vector_list(X[i]),max_length=20) for i in range(X.shape[0])])
# vec_list = vec.padding_truncate_lists(X,max_length=20)

In [13]:
vec_list = np.array(vec_list)

In [14]:
(SequenceLegnth , EmbeddingLength) = vec_list[0].shape

In [15]:
print(f"SequenceLegnth = {SequenceLegnth}, EmbeddingLength = {EmbeddingLength}")

SequenceLegnth = 20, EmbeddingLength = 300


In [16]:
np.save('/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/data/X_vec.npy',vec_list)

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM , Bidirectional , Dropout
#to_categorical
from tensorflow.keras.utils import to_categorical

2022-07-27 09:24:46.986929: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 09:24:46.986990: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
y = to_categorical(y)

In [19]:
model = Sequential()
model.add(Bidirectional(LSTM(units=128, return_sequences=True), input_shape=(SequenceLegnth, EmbeddingLength)))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
model.summary()

2022-07-27 09:24:51.002799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-27 09:24:51.003302: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 09:24:51.003720: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-27 09:24:51.003984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-27 09:24:51.004243: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 20, 256)          439296    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 20, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
Total params: 1,227,776
Trainable params: 1,227,776
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.add(Dense(units=3, activation='softmax'))

In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(vec_list, y, epochs=10, batch_size=32,validation_split=0.2)

2022-07-27 09:24:52.118559: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 67200000 exceeds 10% of free system memory.


88/88 [==============================] - 21s 150ms/step - loss: 0.9123 - accuracy: 0.5532 - val_loss: 0.7770 - val_accuracy: 0.6714
Epoch 2/10
88/88 [==============================] - 12s 134ms/step - loss: 0.7553 - accuracy: 0.6793 - val_loss: 0.7434 - val_accuracy: 0.6900
Epoch 3/10
88/88 [==============================] - 11s 126ms/step - loss: 0.7044 - accuracy: 0.7093 - val_loss: 0.7780 - val_accuracy: 0.6929
Epoch 4/10
88/88 [==============================] - 11s 129ms/step - loss: 0.6431 - accuracy: 0.7443 - val_loss: 0.8105 - val_accuracy: 0.6500
Epoch 5/10
88/88 [==============================] - 10s 116ms/step - loss: 0.5942 - accuracy: 0.7625 - val_loss: 0.7549 - val_accuracy: 0.6886
Epoch 6/10
88/88 [==============================] - 10s 118ms/step - loss: 0.5240 - accuracy: 0.7954 - val_loss: 0.7965 - val_accuracy: 0.6843
Epoch 7/10
88/88 [==============================] - 11s 127ms/step - loss: 0.4616 - accuracy: 0.8279 - val_loss: 0.9214 - val_accuracy: 0.6686
Epoch 8/10

In [23]:
model.evaluate(vec_list, y)

Epoch 1/10
  3/110 [..............................] - ETA: 3s - loss: 0.2392 - accuracy: 0.9271

2022-07-27 09:26:54.989981: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 84000000 exceeds 10% of free system memory.


110/110 [==============================] - 4s 36ms/step - loss: 0.4910 - accuracy: 0.8420


[0.49099525809288025, 0.8420000076293945]

In [24]:
from Modules.text import Preprocessing
prpcs = Preprocessing(lemmatize=True)

In [25]:
tweet  = ["I am a good person in this society and life for me is beautiful"]
tweet = prpcs.text_normalize(tweet[0])
# type(tweet[0])

In [26]:
tweet = prpcs.text_tokenize(tweet)

In [27]:
tweet

['good', 'person', 'society', 'life', 'beautiful']

In [28]:
tweet = vec._get_vector_list(tweet)
tweet = tweet.reshape(1,tweet.shape[0],tweet.shape[1])

In [29]:
tweet.shape

(1, 5, 300)

In [30]:
tweet = vec.padding_truncate_lists(tweet, max_length=20)

In [31]:
tweet.shape

(1, 20, 300)

In [32]:
np.argmax(model.predict(tweet))

1/1 [==============================] - 2s 2s/step


2

In [33]:
model.save('/media/senju/1.0 TB Hard Disk/NLP/models/Tweet_Positive_Negative.h5')